Read SDC Platinum data

In [2]:
import pandas as pd
from  path_utils import get_base_path
import os

# Set path
base = get_base_path()
relative =  r"05 Analysis\01 Main\01 Stata\raw\sdc_gafam\SDC Platinum sdc_gafam_usonly_completed.xlsx"
file_path = os.path.join(base, relative)

# Load the Excel file
df = pd.read_excel(file_path, sheet_name="Request 7", header=2)


Read Target assignees file (JSON)

In [28]:
import json
import os
# Load existing target_assignees_dict from a file if it exists
target_assignees_path = os.path.join(base, r"05 Analysis\01 Main\00 Python data\target_assignees.json")
if os.path.exists(target_assignees_path):
    with open(target_assignees_path, "r") as file:
        try:
            target_assignees_dict = json.load(file)
        except json.JSONDecodeError:
            # In case the file is empty or not properly formatted
            target_assignees_dict = {}
else:
    # Initialize an empty dictionary if the file doesn't exist
    target_assignees_dict = {}


Clean to get create a GAFAM deals only 

In [36]:
# Define GAFAM companies
gafam_companies = ["Google", "Microsoft", "Facebook", "Apple", "Amazon"]

# Create GAFAM variable
df['GAFAM'] = df['Acquiror Full Name'].apply(lambda x: 1 if any(company.lower() in x.lower() for company in gafam_companies) else 0)

# Get TargetFullName for GAFAM == 1
gafam_deals = df[(df['GAFAM'] == 1) & (df['Acquiror Full Name'] != df['Target Full Name'])]['Target Full Name']
gafam_deals = gafam_deals

# Initialize an empty dictionary to store the results
target_assignees_dict = {}


Get best matches from PatensView API

In [ ]:
import api.get_closest_assignees as gca
import importlib
importlib.reload(gca)

import re

# Total number of targets (counting those already in the dictionary)
total_targets = len(gafam_deals)

# Initialize the counter for processed targets
processed_count = len(target_assignees_dict)


# Iterate over each target and apply the get_closest_assignees function
for i, target in enumerate(gafam_deals):
    # Clean target name by removing anything in parentheses (including parentheses)
    cleaned_target = re.sub(r"\(.*?\)", "", target).strip()
    cleaned_target = re.sub(r"\{.*?\}", "", cleaned_target).strip()

    # Check if the cleaned target already exists in the saved dictionary
    if cleaned_target in target_assignees_dict:
        print(f"Skipping {cleaned_target} as it already exists in the saved file.")
        processed_count += 1
        continue

    # Get the closest assignees for the cleaned target
    best_matches = gca.get_closest_assignees(cleaned_target, threshold=75)
    
    # Store the result in the dictionary
    target_assignees_dict[cleaned_target] = {}
    target_assignees_dict[cleaned_target]['str_matched'] = best_matches if best_matches else []
    
    # Update the processed count
    processed_count += 1

    # Print progress every 25 targets
    if processed_count % 25 == 0:
        print(f"Have completed {processed_count}/{total_targets} targets.")


Save the best matches as dict.

In [43]:
# Save the updated target_assignees_dict back to the JSON file
with open(target_assignees_path, "w") as outfile:
    json.dump(target_assignees_dict, outfile, indent=4)

In [34]:
empty_match_count = 0
for key, value in target_assignees_dict.items():
    if 'str_matched' in value and isinstance(value['str_matched'], list) and value['str_matched']:
        empty_match_count += 1

print(f"Number of keys where 'str_matched' is an empty list: {empty_match_count}")


Number of keys where 'str_matched' is an empty list: 311


Save **treated** assignees

In [24]:
import json
import pandas as pd
import os
from path_utils import get_base_path
# File paths
excel_file_path = os.path.join(get_base_path(),r"05 Analysis\01 Main\api\treated_assignees.xlsx")

# Load the JSON data
with open(target_assignees_path, "r") as file:
    target_assignees_dict = json.load(file)

# Collect the data: target company names and matched assignee names
data_to_append = []

for target, value in target_assignees_dict.items():
    # Ensure 'str_matched' exists and is a list of tuples
    if 'str_matched' in value and isinstance(value['str_matched'], list):
        matches = value['str_matched']
        for match_tuple in matches:
            assignee_name = match_tuple[0]  # Get the first item of the tuple (the matched assignee name)
            data_to_append.append({'Target Company': target, 'Assignees': assignee_name})

# Create a DataFrame from the collected data
df_to_append = pd.DataFrame(data_to_append)

# Check if the Excel file already exists
if os.path.exists(excel_file_path):
    # Load existing data
    existing_df = pd.read_excel(excel_file_path)
    # Concatenate the new data with the existing data
    updated_df = pd.concat([existing_df, df_to_append], ignore_index=True)
else:
    # If the file doesn't exist, the updated data is just the new data
    updated_df = df_to_append

# Save the updated DataFrame to the Excel file
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='w') as writer:
    updated_df.to_excel(writer, index=False, sheet_name='Sheet1')

print("Data has been successfully appended to 'treated_assignees.xlsx'.")

Data has been successfully appended to 'treated_assignees.xlsx'.


**Double check the matches with Google Search**

In [40]:
totalcount = 0
for key, value in target_assignees_dict.items():
    if 'str_matched' in value and isinstance(value['str_matched'], list) and value['str_matched']:
        #print(value['str_matched'])
        totalcount += 1

print(f"Number of keys where 'str_matched' is an non-empty list: {totalcount}")

Number of keys where 'str_matched' is an non-empty list: 197


In [41]:
from api.google_search_checker import perform_google_search

local_counter = 0

## Run over the target assignees dict to run the Google Search
for key, value in target_assignees_dict.items():

    ## Skip the loop if no match exists
    if not value['str_matched']:
        continue
    
    ## Get the name  of the target 
    target = key

    ## Get the name of the matches names in a list
    matched_names = [x[0] for x in value['str_matched'] if len(value['google_matched']) == 0]
    
    ## Perform the google search
    google_matches, apicallcount = perform_google_search(target, matched_names)

    ## Assign back to the original dictionary
    value['google_matched'] = google_matches

    ## Print progress
    local_counter += 1
    print(f"Google search progress : {local_counter/totalcount}")

    ## Print API counts
    if apicallcount % 50 == 0:
        print("API call count is: ", apicallcount)



A Saved API
It took 0.56 secs to find the successfull match.
Google search progress : 0.005076142131979695
A Saved API
It took 0.43 secs to find the successfull match.
Google search progress : 0.01015228426395939
A Saved API
It took 0.46 secs to find the successfull match.
Google search progress : 0.015228426395939087
A Saved API
It took 3.28 secs to find the successfull match.
Google search progress : 0.02030456852791878
A Saved API
It took 1.99 secs to find the successfull match.
Google search progress : 0.025380710659898477
It took 0.83 secs to find the successfull match.
Google search progress : 0.030456852791878174
A Saved API
A Saved API
It took 0.42 secs to find the successfull match.
Google search progress : 0.03553299492385787
A Saved API
It took 0.47 secs to find the successfull match.
Google search progress : 0.04060913705583756
It took 3.02 secs to find the successfull match.
Google search progress : 0.04568527918781726
A Saved API
It took 0.59 secs to find the successfull 

**save to an excel**

In [45]:
import json
import pandas as pd
import os
from path_utils import get_base_path
# File paths
excel_file_path = os.path.join(get_base_path(),r"05 Analysis\01 Main\00 Python data\True Matches by Google.xlsx")

# Load the JSON data
with open(target_assignees_path, "r") as file:
    target_assignees_dict = json.load(file)

# Collect the data: target company names and matched assignee names
data_to_append = []

for target, value in target_assignees_dict.items():
    # Ensure 'str_matched' exists and is a list of tuples
    if 'google_matched' in value and isinstance(value['google_matched'], list):
        matches = value['google_matched']
        for match in matches:
            data_to_append.append({'Target Company': target, 'Assignees': match})

# Create a DataFrame from the collected data
df_to_append = pd.DataFrame(data_to_append)

# Check if the Excel file already exists
if os.path.exists(excel_file_path):
    # Load existing data
    existing_df = pd.read_excel(excel_file_path)
    # Concatenate the new data with the existing data
    updated_df = pd.concat([existing_df, df_to_append], ignore_index=True)
else:
    # If the file doesn't exist, the updated data is just the new data
    updated_df = df_to_append

# Save the updated DataFrame to the Excel file
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='w') as writer:
    updated_df.to_excel(writer, index=False, sheet_name='cleaned')

print("Data has been successfully appended to 'treated_assignees.xlsx'.")

Data has been successfully appended to 'treated_assignees.xlsx'.
